In [16]:
from utils import get_spark_session

In [17]:
spark = get_spark_session("iceberg", "landing")
print(spark)

In [19]:
df = spark.read.csv("s3a://landing/marca_carro.csv", header=True, inferSchema=True)
df.show()

+------------+---------+
| marca_carro|cod_marca|
+------------+---------+
|       Acura|        1|
|Aston Martin|        2|
|        Audi|        3|
|      Austin|        4|
|         BMW|        5|
|     Bentley|        6|
|     Bugatti|        7|
|       Buick|        8|
|    Cadillac|        9|
|   Chevrolet|       10|
|    Chrysler|       11|
|     Citroën|       12|
|      Daewoo|       13|
|       Dodge|       14|
|       Eagle|       15|
|     Ferrari|       16|
|        Ford|       17|
|         GMC|       18|
|         Geo|       19|
|       Honda|       20|
+------------+---------+
only showing top 20 rows



In [15]:
spark.stop()